<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Maximiliano Quintero Ortega
- Nombre de alumno 2: Tomás Apablaza Arriagada


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/Repositivo/Repositivo.git`

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.4 MB/s eta 0:00:00


# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

<ipython-input-26-a738af29f7fb>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [ ]:
# Librerías
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib


# 1) Dividir los datos en train, validation y test
labels = df.loc[:, "quantity"]
features = df.drop(columns=["quantity","id"])

X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.3, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

#2)
def extract_date_features(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)
date_transformer = FunctionTransformer(extract_date_features)

#3)
categorical_features = ["city","shop","brand","container","capacity"]
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

#4)
pipeline_dummy = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
])

#5)
pipeline_dummy.fit(X_train, y_train)
preds = pipeline_dummy.predict(X_validation)
mae_dummy = mean_absolute_error(y_validation, preds)
print(f"MAE Dummy: {mae_dummy}")

#6)
pipeline_xgb = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])
pipeline_xgb.fit(X_train, y_train)
preds_xgb = pipeline_xgb.predict(X_validation)
mae_xgb = mean_absolute_error(y_validation, preds_xgb)
print(f"MAE XGB: {mae_xgb}")

#7)
joblib.dump(pipeline_dummy, 'pipeline_dummy.pkl')
joblib.dump(pipeline_xgb, 'pipeline_xgb.pkl')

MAE Dummy: 13308.134750658153
MAE XGB: 8231.33520475217


['pipeline_xgb.pkl']

5. Esta métrica permite observar la desviación promedio de la demanda predicha en comparación a la demanda real.
6. Al utilizar XGB disminuye el MAE en comparación con Dummy Regressor, lo que indica una mejora en el modelo.

## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




In [ ]:
#1)
monotone_constraints = (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1)

pipeline_xgb_monotonic = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints = monotone_constraints))
])
pipeline_xgb_monotonic.fit(X_train, y_train)
preds_xgb_monotonic = pipeline_xgb_monotonic.predict(X_validation)
#2)
mae_xgb_monotonic = mean_absolute_error(y_validation, preds_xgb_monotonic)
print(f"MAE XGB Monotonic: {mae_xgb_monotonic}")

MAE XGB Monotonic: 8231.335395110147


El MAE utilizando esta restricción es casi identico al anterior.

## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [ ]:
import optuna
from optuna.samplers import TPESampler
import pandas as pd

# Asegurar reproducibilidad
SEED = 42

# Definir la función objetiva para Optuna
def objective(trial):
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0, 1)
    reg_lambda = trial.suggest_float("reg_lambda", 0, 1)
    min_frequency = trial.suggest_float("min_frequency", 0.0, 1.0)

    # Definir el preprocesador con el hiperparámetro min_frequency
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(min_frequency=min_frequency), categorical_features)
        ])

    # Definir el pipeline con XGBRegressor y los hiperparámetros optimizados
    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            random_state=SEED
        ))
    ])

    # Entrenar el pipeline
    pipeline_xgb.fit(X_train, y_train)

    # Predecir y calcular el MAE
    preds_xgb = pipeline_xgb.predict(X_validation)
    mae_xgb = mean_absolute_error(y_validation, preds_xgb)

    return mae_xgb

# Crear estudio con TPESampler
study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=SEED))
# Limitar el tiempo de entrenamiento a 5 minutos (300 segundos)
study.optimize(objective, timeout=300)

# Resultados
best_params = study.best_params
best_mae = study.best_value
n_trials = len(study.trials)

print(f"Number of trials: {n_trials}")
print(f"Best MAE: {best_mae}")
print(f"Best hyperparameters: {best_params}")

# Guardar el mejor modelo
best_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(min_frequency=best_params['min_frequency']), categorical_features)
        ])),
    ('regressor', XGBRegressor(
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        max_leaves=best_params['max_leaves'],
        min_child_weight=best_params['min_child_weight'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        random_state=SEED
    ))
])
best_pipeline.fit(X_train, y_train)
joblib.dump(best_pipeline, 'best_pipeline_xgb.pkl')

[I 2024-06-03 01:46:18,814] A new study created in memory with name: no-name-49591e56-e78c-4b83-a835-90101f8db3f4
[I 2024-06-03 01:46:19,289] Trial 0 finished with value: 13308.134690003339 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352}. Best is trial 0 with value: 13308.134690003339.
[I 2024-06-03 01:46:19,662] Trial 1 finished with value: 9138.077934642022 and parameters: {'learning_rate': 0.06051038616257767, 'n_estimators': 723, 'max_depth': 3, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frequency': 0.18340450985343382}. Best is trial 1 with value: 9138.077934642022.
[I 2024-06-03 01:46:19,817] Trial 2 finished with value: 13308.134690003339 and parameters: {'learning_rate': 0.03111998205299424, 'n_estimators': 549, 'max_de

Number of trials: 606
Best MAE: 8202.776103163593
Best hyperparameters: {'learning_rate': 0.05932145575752604, 'n_estimators': 630, 'max_depth': 3, 'max_leaves': 3, 'min_child_weight': 3, 'reg_alpha': 0.950066196395055, 'reg_lambda': 0.9453963499802847, 'min_frequency': 0.05284633143962786}


['best_pipeline_xgb.pkl']

Explicación de Parámetros:
- learning_rate: Tasa de aprendizaje del modelo.
- n_estimators: Número de árboles en el conjunto.
- max_depth: Profundidad máxima de cada árbol.
- max_leaves: Número máximo de hojas en los árboles.
- min_child_weight: Peso mínimo de las hojas en el árbol.
- reg_alpha y reg_lambda: Parámetros de regularización L1 y L2, respectivamente.
- min_frequency: Frecuencia mínima para crear categorías en OneHotEncoder.

El resultado mejora en comparción al baseline y XGB sin optimizar, se disminuye le mae en 100 unidades lo que resulta en predicciones más precisas.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 928.0 kB/s eta 0:00:00


In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
import pandas as pd
from xgboost import XGBRegressor
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

# Asegurar reproducibilidad
SEED = 42

# Supongamos que date_transformer, categorical_features, X_train, y_train, X_validation, y_validation ya están definidos

def pipeline_fit_with_eval_set(pipeline, X_train, y_train, X_test, y_test, fit_params={}):
    """
    Fit a scikit-learn pipeline with eval_set support.

    Parameters:
    - pipeline: The scikit-learn pipeline.
    - X_train: Training data.
    - y_train: Training labels.
    - X_test: Test data.
    - y_test: Test labels.
    - fit_params: Additional fit parameters.
    - pipeline_model_step_name: Name of the model step in the pipeline.

    Usage:
    pipeline_fit_with_eval_set(my_pipeline, X_train, y_train, X_test, y_test, fit_params={'eval_metric': 'logloss'})
    """
    # Step 1: Extract Preprocessors
    pipeline_preprocessors = Pipeline(pipeline.steps[:-1])

    # Step 2: Fit preprocessors and Transform Training Data
    # Make sure not to use any test data for the fit step
    X_train_transformed = pipeline_preprocessors.fit_transform(X_train)

    # Step 3: Transform Test Data
    X_test_transformed = pipeline_preprocessors.transform(X_test)

    # Step 4: Prepare Eval Set
    fit_params["eval_set"] = [(X_test_transformed, y_test)]

    # Step 5: Extract Model and Fit
    model = pipeline.steps[-1][1]
    model.fit(X_train_transformed, y_train, **fit_params)

# Definir la función objetiva para Optuna
def objective(trial):
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0, 1)
    reg_lambda = trial.suggest_float("reg_lambda", 0, 1)
    min_frequency = trial.suggest_float("min_frequency", 0.0, 1.0)

    # Definir el preprocesador con el hiperparámetro min_frequency

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_features)
        ], remainder='passthrough')

    # Definir el pipeline con XGBRegressor y los hiperparámetros optimizados
    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            enable_categorical=True,  # Habilitar soporte para categóricos
            random_state=SEED
        ))
    ])

    pipeline_fit_with_eval_set(pipeline_xgb, X_train, y_train, X_validation, y_validation, fit_params = {'callbacks': [XGBoostPruningCallback(trial, "validation_0-rmse")]})
    # Entrenar el pipeline con callback de pruning

    # Predecir y calcular el MAE
    preds_xgb = pipeline_xgb.predict(X_validation)
    mae_xgb = mean_absolute_error(y_validation, preds_xgb)

    return mae_xgb

# Crear estudio con TPESampler
study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=SEED))
# Limitar el tiempo de entrenamiento a 5 minutos (300 segundos)
study.optimize(objective, timeout=300, show_progress_bar=True)

# Resultados
best_params = study.best_params
best_mae = study.best_value
n_trials = len(study.trials)

print(f"Number of trials: {n_trials}")
print(f"Best MAE: {best_mae}")
print(f"Best hyperparameters: {best_params}")

# Guardar el mejor modelo
best_pipeline = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(min_frequency=best_params['min_frequency']), categorical_features)
        ])),
    ('regressor', XGBRegressor(
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        max_leaves=best_params['max_leaves'],
        min_child_weight=best_params['min_child_weight'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        enable_categorical=True,  # Habilitar soporte para categóricos
        random_state=SEED
    ))
])
best_pipeline.fit(X_train, y_train)
joblib.dump(best_pipeline, 'best_pipeline_xgb_pruned.pkl')

[I 2024-06-03 01:52:33,293] A new study created in memory with name: no-name-4bb1bf15-d8ef-42c2-a307-a070d52708b1


   0%|          | 00:00/05:00

[0]	validation_0-rmse:17159.66855
[1]	validation_0-rmse:16713.98551
[2]	validation_0-rmse:16294.91249
[3]	validation_0-rmse:15897.26861
[4]	validation_0-rmse:15506.24357
[5]	validation_0-rmse:15131.92777
[6]	validation_0-rmse:14764.03941
[7]	validation_0-rmse:14422.61607
[8]	validation_0-rmse:14103.38473
[9]	validation_0-rmse:13783.38929
[10]	validation_0-rmse:13465.14298
[11]	validation_0-rmse:13170.94379
[12]	validation_0-rmse:12889.01832
[13]	validation_0-rmse:12616.54963
[14]	validation_0-rmse:12343.13185
[15]	validation_0-rmse:12092.02465
[16]	validation_0-rmse:11849.51533
[17]	validation_0-rmse:11618.98549
[18]	validation_0-rmse:11385.11054
[19]	validation_0-rmse:11161.68697
[20]	validation_0-rmse:10950.00137
[21]	validation_0-rmse:10743.11364


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[22]	validation_0-rmse:10555.82255
[23]	validation_0-rmse:10359.32665
[24]	validation_0-rmse:10176.50918
[25]	validation_0-rmse:10005.58026
[26]	validation_0-rmse:9826.37453
[27]	validation_0-rmse:9677.81629
[28]	validation_0-rmse:9519.42070
[29]	validation_0-rmse:9353.98453
[30]	validation_0-rmse:9204.06987
[31]	validation_0-rmse:9063.49406
[32]	validation_0-rmse:8926.34888
[33]	validation_0-rmse:8782.58861
[34]	validation_0-rmse:8644.59087
[35]	validation_0-rmse:8526.30492
[36]	validation_0-rmse:8410.19086
[37]	validation_0-rmse:8284.35759
[38]	validation_0-rmse:8166.87902
[39]	validation_0-rmse:8053.51615
[40]	validation_0-rmse:7946.31964
[41]	validation_0-rmse:7845.19467
[42]	validation_0-rmse:7743.22651
[43]	validation_0-rmse:7652.76435
[44]	validation_0-rmse:7555.84693
[45]	validation_0-rmse:7466.50073
[46]	validation_0-rmse:7378.52829
[47]	validation_0-rmse:7288.06905
[48]	validation_0-rmse:7200.21551
[49]	validation_0-rmse:7103.97415
[50]	validation_0-rmse:7029.46900
[51]	valid

...

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[12]	validation_0-rmse:8660.22241
[13]	validation_0-rmse:8292.09227
[14]	validation_0-rmse:7981.38438
[15]	validation_0-rmse:7679.85063
[16]	validation_0-rmse:7422.26086
[17]	validation_0-rmse:7165.50105
[18]	validation_0-rmse:6938.21612
[19]	validation_0-rmse:6740.69127
[20]	validation_0-rmse:6546.40625
[21]	validation_0-rmse:6373.47606
[22]	validation_0-rmse:6214.66607
[23]	validation_0-rmse:6069.99428
[24]	validation_0-rmse:5939.45959
[25]	validation_0-rmse:5783.74501
[26]	validation_0-rmse:5666.74471
[27]	validation_0-rmse:5554.17126
[28]	validation_0-rmse:5430.66688
[29]	validation_0-rmse:5325.69212
[30]	validation_0-rmse:5213.71650
[31]	validation_0-rmse:5124.59072
[32]	validation_0-rmse:5059.81214
[33]	validation_0-rmse:4985.19710
[34]	validation_0-rmse:4917.22121
[35]	validation_0-rmse:4844.37201
[36]	validation_0-rmse:4785.91217
[37]	validation_0-rmse:4721.88268
[38]	validation_0-rmse:4685.54972
[39]	validation_0-rmse:4651.86310
[40]	validation_0-rmse:4603.88171
[41]	validatio

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[17]	validation_0-rmse:8029.96152
[18]	validation_0-rmse:7770.84497
[19]	validation_0-rmse:7558.12062
[20]	validation_0-rmse:7333.80647
[21]	validation_0-rmse:7115.69299
[22]	validation_0-rmse:6955.31249
[23]	validation_0-rmse:6791.44125
[24]	validation_0-rmse:6630.64130
[25]	validation_0-rmse:6475.51967
[26]	validation_0-rmse:6329.42506
[27]	validation_0-rmse:6199.18408
[28]	validation_0-rmse:6067.61759
[29]	validation_0-rmse:5950.20301
[30]	validation_0-rmse:5843.09410
[31]	validation_0-rmse:5723.16543
[32]	validation_0-rmse:5613.35132
[33]	validation_0-rmse:5530.29332
[34]	validation_0-rmse:5446.96881
[35]	validation_0-rmse:5372.66074
[36]	validation_0-rmse:5301.06414
[37]	validation_0-rmse:5208.52931
[38]	validation_0-rmse:5122.10103
[39]	validation_0-rmse:5068.00972
[40]	validation_0-rmse:4991.31612
[41]	validation_0-rmse:4927.95288
[42]	validation_0-rmse:4862.36252
[43]	validation_0-rmse:4805.40465
[44]	validation_0-rmse:4752.27664
[45]	validation_0-rmse:4703.33466
[46]	validatio

['best_pipeline_xgb_pruned.pkl']

In [ ]:
# Resultados
best_params = study.best_params
best_mae = study.best_value
n_trials = len(study.trials)

print(f"Number of trials: {n_trials}")
print(f"Best MAE: {best_mae}")
print(f"Best hyperparameters: {best_params}")

Number of trials: 45
Best MAE: 1984.5165295263475
Best hyperparameters: {'learning_rate': 0.08575135589203793, 'n_estimators': 990, 'max_depth': 8, 'max_leaves': 85, 'min_child_weight': 3, 'reg_alpha': 0.5917391650064411, 'reg_lambda': 0.6823814973222213, 'min_frequency': 0.4508120824336694}


- Poda (Pruning) XGBoostPruningCallback: Callback que permite la poda durante la optimización para detener evaluaciones que no están mostrando mejoras prometedoras, lo que ayuda a reducir el tiempo de entrenamiento.

El MAE mejora considerablemente en comparación a los demás modelos, esto puede deberse a que como detiene evaluaciones que no tienen una mejora prometedora, dentro de los 5 mins tiene más tiempo para evaluar posibilidades de los hiperparametros y encontrar los más adecuados.

## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>